In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from generate_colormap import *

from scipy.io import readsav

import jdcal as j 
from datetime import date

import pandas as pd

In [ ]:
#Path for simulation results
path = '/home/sophie-stucki/starsim/starsim/SDO_input/flux_output/'
#Path of the classification maps
path_maps = '/home/sophie-stucki/starsim/starsim/SDO_input/maps/'
#Folder name
folder = '3_years'

#Timeline of the simulations
start_time = j.gcal2jd(2015,7,1)[1] + 2400000.5
end_time = j.gcal2jd(2018,7,1)[1] + 2400000.5

#Load simulation data
simu_mps = np.loadtxt(path + '{}/flux_test_mps.txt'.format(folder))
simu_phoenix = np.loadtxt(path + '{}/flux_test_phoenix.txt'.format(folder))

days_simu = np.loadtxt(path_maps + '{}/days_list.txt'.format(folder))

#Add STAGGER
stagger = False

if stagger:
    simu_stagger = np.loadtxt(path + '{}/flux_test_stagger.txt'.format(folder))

#Use SORCE/TIM data
sorce = True

if sorce:
    sorce_data = readsav('/home/sophie-stucki/Documents/starsim_simulations/nina_spec/DataSSI-SORCE.sav')

#Use TSIS data
tsis_path = None

if tsis_path != None:
    data_tsis = pd.read_csv('/home/sophie-stucki/starsim/starsim/SDO_input/tsis_tsi_24hr_2018.csv')


In [ ]:
#Collect the SORCE data
if sorce:
    t_tsi = []
    data_tsi = []

    days_tsis = data_tsis['time (Julian Date)'] - start_time

    for i in range(len(sorce_data['time'])):
        if sorce_data['time'][i] >= start_time and sorce_data['time'][i] <= end_time:
            t_tsi.append(j.jd2gcal(2400000.5,sorce_data['time'][i]-2400000.5))

            data_tsi.append(sorce_data['tsi'][i])


    data_tsi = np.squeeze(data_tsi)
    data_tsi *= 100**2 * 10000000

    # Date de départ
    start_date = date(t_tsi[0][0], t_tsi[0][1], t_tsi[0][2])

    # Calcul du nombre de jours pour chaque date
    days_tsi = [(date(y, m, d) - start_date).days + f for (y, m, d, f) in t_tsi]

#Mask the zero-values in TSIS data
if tsis_path != None:
    mask_tsis = data_tsis['tsi_1au (W/m^2)']>0
    data_tsis = mask_tsis[mask_tsis]
    days_tsis = days_tsis[mask_tsis]

#Residues
if sorce:
    days_res_tsi = []
    res_tsi_mps = []
    res_tsi_phoenix = []
    rel_res_tsi_mps = []
    rel_res_tsi_phoenix = []

    if stagger:
        res_tsi_stagger = []
        rel_res_tsi_stagger = []


    for i, d in enumerate(days_simu):
        for j, k in enumerate(days_tsi):
            if d == k:
                days_res_tsi.append(d)
                res_tsi_mps.append(1e6 * (data_tsi[j] / np.mean(data_tsi) - simu_mps[i] / np.mean(simu_mps)))
                rel_res_tsi_mps.append(100 * (data_tsi[j] / np.mean(data_tsi) - simu_mps[i] / np.mean(simu_mps)) / (data_tsi[j] / np.mean(data_tsi)))

                res_tsi_phoenix.append(1e6 * (data_tsi[j] / np.mean(data_tsi) - simu_phoenix[i] / np.mean(simu_phoenix)))
                rel_res_tsi_phoenix.append(100 * (data_tsi[j] / np.mean(data_tsi) - simu_phoenix[i] / np.mean(simu_phoenix)) / (data_tsi[j] / np.mean(data_tsi)))

                if stagger:
                    res_tsi_stagger.append(1e6 * (data_tsi[j] / np.mean(data_tsi) - simu_stagger[i] / np.mean(simu_stagger)))
                    rel_res_tsi_stagger.append(100 * (data_tsi[j] / np.mean(data_tsi) - simu_stagger[i] / np.mean(simu_stagger)) / (data_tsi[j] / np.mean(data_tsi)))

if tsis_path != None:
    days_res_tsis = []
    res_tsis_mps = []
    res_tsis_phoenix = []
    rel_res_tsis_mps = []
    rel_res_tsis_phoenix = []

    if stagger:
        res_tsis_stagger = []
        rel_res_tsis_stagger = []


    for i, d in enumerate(days_simu):
        for j, k in enumerate(days_tsis):
            if d == k:
                days_res_tsis.append(d)
                res_tsis_mps.append(1e6 * (data_tsis[j] / np.mean(data_tsis) - simu_mps[i] / np.mean(simu_mps)))
                rel_res_tsis_mps.append(100 * (data_tsis[j] / np.mean(data_tsis) - simu_mps[i] / np.mean(simu_mps)) / (data_tsis[j] / np.mean(data_tsis)))

                res_tsis_phoenix.append(1e6 * (data_tsis[j] / np.mean(data_tsis) - simu_phoenix[i] / np.mean(simu_phoenix)))
                rel_res_tsis_phoenix.append(100 * (data_tsis[j] / np.mean(data_tsis) - simu_phoenix[i] / np.mean(simu_phoenix)) / (data_tsis[j] / np.mean(data_tsis)))

                if stagger:
                    res_tsis_stagger.append(1e6 * (data_tsis[j] / np.mean(data_tsis) - simu_stagger[i] / np.mean(simu_stagger)))
                    rel_res_tsis_stagger.append(100 * (data_tsis[j] / np.mean(data_tsis) - simu_stagger[i] / np.mean(simu_stagger)) / (data_tsis[j] / np.mean(data_tsis)))



In [ ]:
###Ploting the photometetric benchmark

#Comparison
plt.rcParams['font.size'] = 18
plt.rcParams['figure.figsize'] = (12,7)

if tsis_path != None:
    plt.plot(days_tsis, 1e6 * data_tsis['tsi_1au (W/m^2)'] / np.mean(data_tsis['tsi_1au (W/m^2)']) - 1e6 , '-', label='TSIS', color="#76C0FD", alpha =0.3, linewidth=2)
if sorce:
    plt.plot(days_tsi, 1e6 * data_tsi / np.mean(data_tsi) - 1e6 , '-', label='TSI: SORCE/TIM', color="#000000", alpha =0.3, linewidth=2)
plt.plot(days_simu, 1e6 * simu_mps / np.mean(simu_mps) - 1e6,'s--', markersize=5, linewidth=1.7, label = 'SunSim: MPS-ATLAS', color='#ef946c', alpha=0.7)
if stagger:
    plt.plot(days_simu, 1e6 * simu_stagger / np.mean(simu_stagger) - 1e6,'s-', markersize=5, linewidth=1.7, label = 'SunSim: STAGGER', color="#ce9ffa", alpha=0.7)
plt.plot(days_simu, 1e6 * simu_phoenix / np.mean(simu_phoenix) - 1e6,'s--', markersize=5, linewidth=1.7, label = 'SunSim: PHOENIX', color="#a4d485", alpha=0.4)

plt.legend(fontsize=12)
plt.xlabel('t [day]')

plt.ylabel('intensity variability [ppm]')
plt.savefig(path + 'photometric_comparison.png',  dpi=300, bbox_inches='tight')

plt.show()

#Residues
if sorce:
    plt.axhline(0, linestyle= '--', color="#000000", alpha =0.3, linewidth=2 )
    plt.plot(days_res_tsi, res_tsi_mps ,'s--', markersize=5, linewidth=1.7, label = 'SunSim: MPS-ATLAS', color='#ef946c', alpha=0.7)
    plt.plot(days_res_tsi, res_tsi_phoenix,'s--', markersize=5, linewidth=1.7, label = 'SunSim: PHOENIX', color="#a4d485", alpha=0.4)
    if stagger:
        plt.plot(days_res_tsi, res_tsi_stagger,'s-', markersize=5, linewidth=1.7, label = 'SunSim: STAGGER', color="#ce9ffa", alpha=0.7)

    plt.title('SORCE/TIM')
    plt.legend(fontsize=12)
    plt.xlabel('t [day]')
    plt.ylabel('Residues [ppm]')
    plt.savefig(path + 'photometric_residues_SORCE.png',  dpi=300, bbox_inches='tight')

    plt.show()

if tsis_path != None:
    plt.axhline(0, linestyle= '--', color="#000000", alpha =0.3, linewidth=2 )
    plt.plot(days_res_tsis, res_tsis_mps ,'s--', markersize=5, linewidth=1.7, label = 'SunSim: MPS-ATLAS', color='#ef946c', alpha=0.7)
    plt.plot(days_res_tsis, res_tsis_phoenix,'s--', markersize=5, linewidth=1.7, label = 'SunSim: PHOENIX', color="#a4d485", alpha=0.4)
    if stagger:
        plt.plot(days_res_tsis, res_tsis_stagger,'s-', markersize=5, linewidth=1.7, label = 'SunSim: STAGGER', color="#ce9ffa", alpha=0.7)

    plt.title('TSIS/TIM')
    plt.legend(fontsize=12)
    plt.xlabel('t [day]')
    plt.ylabel('Residues [ppm]')
    plt.savefig(path + 'photometric_residues_TSIS.png',  dpi=300, bbox_inches='tight')

    plt.show()

#Relative residues
if sorce:
    plt.axhline(0, linestyle= '--', color="#000000", alpha =0.3, linewidth=2 )
    plt.plot(days_res_tsi, rel_res_tsi_mps,'s--', markersize=5, linewidth=1.7, label = 'SunSim: MPS-ATLAS', color='#ef946c', alpha=0.7)
    plt.plot(days_res_tsi, rel_res_tsi_phoenix,'s--', markersize=5, linewidth=1.7, label = 'SunSim: PHOENIX', color="#a4d485", alpha=0.4)
    if stagger:
        plt.plot(days_res_tsi, rel_res_tsi_stagger,'s-', markersize=5, linewidth=1.7, label = 'SunSim: STAGGER', color="#ce9ffa", alpha=0.7)

    plt.title('SORCE/TIM')
    plt.legend(fontsize=12)
    plt.xlabel('t [day]')
    plt.ylabel('Relative residues [%]')
    plt.savefig(path + 'photometric_rel_residues_SORCE.png',  dpi=300, bbox_inches='tight')

    plt.show()

if tsis_path != None:
    plt.axhline(0, linestyle= '--', color="#000000", alpha =0.3, linewidth=2 )
    plt.plot(days_res_tsis, rel_res_tsis_mps,'s--', markersize=5, linewidth=1.7, label = 'SunSim: MPS-ATLAS', color='#ef946c', alpha=0.7)
    plt.plot(days_res_tsis, rel_res_tsis_phoenix,'s--', markersize=5, linewidth=1.7, label = 'SunSim: PHOENIX', color="#a4d485", alpha=0.4)
    if stagger:
        plt.plot(days_res_tsis, rel_res_tsis_stagger,'s-', markersize=5, linewidth=1.7, label = 'SunSim: STAGGER', color="#ce9ffa", alpha=0.7)

    plt.title('TSIS/TIM')
    plt.legend(fontsize=12)
    plt.xlabel('t [day]')
    plt.ylabel('Relative residues [%]')
    plt.savefig(path + 'photometric_rel_residues_TSIS.png',  dpi=300, bbox_inches='tight')

    plt.show()